# FCN
This notebook contains a test CNN in pytorch, to get familiar with this developping environment. It also acts as a template for later use.

In [ ]:
%matplotlib inline
import numpy as np
import torch
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import os,sys
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import time

#seed for reproducible results
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

In [ ]:
# Helper functions

def load_image(infilename):
    data = mpimg.imread(infilename)
    return data

def img_float_to_uint8(img):
    rimg = img - np.min(img)
    rimg = (rimg / np.max(rimg) * 255).round().astype(np.uint8)
    return rimg

# Concatenate an image and its groundtruth
def concatenate_images(img, gt_img):
    nChannels = len(gt_img.shape)
    w = gt_img.shape[0]
    h = gt_img.shape[1]
    if nChannels == 3:
        cimg = np.concatenate((img, gt_img), axis=1)
    else:
        gt_img_3c = np.zeros((w, h, 3), dtype=np.uint8)
        gt_img8 = img_float_to_uint8(gt_img)          
        gt_img_3c[:,:,0] = gt_img8
        gt_img_3c[:,:,1] = gt_img8
        gt_img_3c[:,:,2] = gt_img8
        img8 = img_float_to_uint8(img)
        cimg = np.concatenate((img8, gt_img_3c), axis=1)
    return cimg

def img_crop(im, w, h):
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    is_2d = len(im.shape) < 3
    for i in range(0,imgheight,h):
        for j in range(0,imgwidth,w):
            if is_2d:
                im_patch = im[j:j+w, i:i+h]
            else:
                im_patch = im[j:j+w, i:i+h, :]
            list_patches.append(im_patch)
    return list_patches

In [ ]:
from random import random

def uniform(a,b):
    return(a+(b-a)*random())

def img_rnd_crop(im, w, h, i = -1, j = -1):
    is_2d = len(im.shape) < 3
    imgwidth = im.shape[len(im.shape)-2]
    imgheight = im.shape[len(im.shape)-1]
    if (i == -1 and j == -1):
        i = int(uniform(0, imgwidth-w-1))
        j = int(uniform(0, imgheight-h-1))
    if is_2d:
        im_patch = im[i:i+w, j:j+h]
    else:
        im_patch = im[:, i:i+w, j:j+h]
    return im_patch, i, j

## Pytorch module

This module contains the FCN based on the CNN module

### Module structure

For this module, we will try to implement the complex diagram describe in http://openaccess.thecvf.com/content_cvpr_2018_workshops/papers/w4/Zhou_D-LinkNet_LinkNet_With_CVPR_2018_paper.pdf

Usefull links:

-Torch documentations (especially for the input/ouput size of Conv2d and ConvTranspose2d: https://pytorch.org/docs/stable/nn.html

-This link to better understand what each argument in Conv2d and ConvTranspose2d: https://github.com/vdumoulin/conv_arithmetic/blob/master/README.md

In [ ]:
# This class takes our input of size 400*400 and enlarges it to size 512*512

class FCN_DLinkNet(torch.nn.Module):
        
    def __init__(self):
        super(FCN_DLinkNet, self).__init__()
        
        # Encoding part
        
        self.convstart = torch.nn.Conv2d(3, 64, kernel_size=5, stride=1, padding=58)
        self.pool = torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.convEncodBlock1 = torch.nn.Conv2d(64, 64, kernel_size=3, padding=1)
        
        self.convEncodBlock2start = torch.nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.convEncodBlock2 = torch.nn.Conv2d(128, 128, kernel_size=3, padding=1)
        
        self.convEncodBlock3start = torch.nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.convEncodBlock3 = torch.nn.Conv2d(256, 256, kernel_size=3, padding=1)
        
        self.convEncodBlock4start = torch.nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.convEncodBlock4 = torch.nn.Conv2d(512, 512, kernel_size=3, padding=1)
        
        # Center part
        
        self.convCenterdil1 = torch.nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, dilation=1) #same as convEncodBlock4 but added for clarity
        self.convCenterdil2 = torch.nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=2, dilation=2)
        self.convCenterdil4 = torch.nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=4, dilation=4)
        self.convCenterdil8 = torch.nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=8, dilation=8)
        
        # Decoding part
        
        self.convDecodBlock4start = torch.nn.Conv2d(512, 128, kernel_size=1)
        self.deconvDecodBlock4 = torch.nn.ConvTranspose2d(128, 128, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.convDecodBlock4end = torch.nn.Conv2d(128, 256, kernel_size=1)
        
        self.convDecodBlock3start = torch.nn.Conv2d(256, 64, kernel_size=1)
        self.deconvDecodBlock3 = torch.nn.ConvTranspose2d(64, 64, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.convDecodBlock3end = torch.nn.Conv2d(64, 128, kernel_size=1)
        
        self.convDecodBlock2start = torch.nn.Conv2d(128, 32, kernel_size=1)
        self.deconvDecodBlock2 = torch.nn.ConvTranspose2d(32, 32, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.convDecodBlock2end = torch.nn.Conv2d(32, 64, kernel_size=1)
        
        self.convDecodBlock1start = torch.nn.Conv2d(64, 16, kernel_size=1)
        self.deconvDecodBlock1 = torch.nn.ConvTranspose2d(16, 16, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.convDecodBlock1end = torch.nn.Conv2d(16, 64, kernel_size=1)
        
        self.deconvDecodend = torch.nn.ConvTranspose2d(64, 32, kernel_size=3, stride=1, padding=57)
        self.convDecoddil1end = torch.nn.Conv2d(32, 2, kernel_size=3, stride=1, padding=1, dilation=1)
        
        self.sigmoid = torch.nn.Sigmoid()
        
        
    def forward(self, x):
        
        def codeBlock1(self, x):
            for i in range(0, 3):
                x_inter = F.relu(self.convEncodBlock1(x))
                x_inter = F.relu(self.convEncodBlock1(x_inter))
                # Moyenne
                x = (x + x_inter)/2
            return x

        def codeBlock2(self, x):
            x = self.pool(x)
            x_inter = F.relu(self.convEncodBlock2start(x))
            x_inter = F.relu(self.convEncodBlock2(x_inter))
            new_x = torch.zeros(x_inter.shape)
            new_x[:, :64, :, :] = x
            # Moyenne
            x = (new_x + x_inter)/2
            for i in range(0, 3):
                x_inter = F.relu(self.convEncodBlock2(x))
                x_inter = F.relu(self.convEncodBlock2(x_inter))
                # Moyenne
                x = (x + x_inter)/2
            return x

        def codeBlock3(self, x):
            x = self.pool(x)
            x_inter = F.relu(self.convEncodBlock3start(x))
            x_inter = F.relu(self.convEncodBlock3(x_inter))
            new_x = torch.zeros(x_inter.shape)
            new_x[:, :128, :, :] = x
            # Moyenne
            x = (new_x + x_inter)/2
            for i in range(0, 5):
                x_inter = F.relu(self.convEncodBlock3(x))
                x_inter = F.relu(self.convEncodBlock3(x_inter))
                # Moyenne
                x = (x + x_inter)/2
            return x

        def codeBlock4(self, x):
            x = self.pool(x)
            x_inter = F.relu(self.convEncodBlock4start(x))
            x_inter = F.relu(self.convEncodBlock4(x_inter))
            new_x = torch.zeros(x_inter.shape)
            new_x[:, :256, :, :] = x
            # Moyenne
            x = (new_x + x_inter)/2
            for i in range(0, 2):
                x_inter = F.relu(self.convEncodBlock4(x))
                x_inter = F.relu(self.convEncodBlock4(x_inter))
                # Moyenne
                x = (x + x_inter)/2
            return x

        def decodeBlock4(self, x):
            x = F.relu(self.convDecodBlock4start(x))
            x = self.deconvDecodBlock4(x)
            x = F.relu(self.convDecodBlock4end(x))
            return x

        def decodeBlock3(self, x):
            x = F.relu(self.convDecodBlock3start(x))
            x = self.deconvDecodBlock3(x)
            x = F.relu(self.convDecodBlock3end(x))
            return x

        def decodeBlock2(self, x):
            x = F.relu(self.convDecodBlock2start(x))
            x = self.deconvDecodBlock2(x)
            x = F.relu(self.convDecodBlock2end(x))
            return x

        def decodeBlock1(self, x):
            x = F.relu(self.convDecodBlock1start(x))
            x = self.deconvDecodBlock1(x)
            x = F.relu(self.convDecodBlock1end(x))
            return x
        
        x_start = F.relu(self.convstart(x))
        x_start = self.pool(x_start)
        
        x_codeBlock1 = codeBlock1(self, x_start)
        x_codeBlock2 = codeBlock2(self, x_codeBlock1)
        x_codeBlock3 = codeBlock3(self, x_codeBlock2)
        x_codeBlock4 = codeBlock4(self, x_codeBlock3)
        
        x_center1 = F.relu(self.convCenterdil1(x_codeBlock4))
        x_center2 = F.relu(self.convCenterdil2(x_center1))
        x_center3 = F.relu(self.convCenterdil4(x_center2))
        x_center4 = F.relu(self.convCenterdil8(x_center3))
        
        x_center_final = (x_codeBlock4 + x_center1 + x_center2 + x_center3 + x_center4)/5
        
        x_decodeBlock4 = decodeBlock4(self, x_center_final)
        x_decodeBlock3 = decodeBlock3(self, (x_decodeBlock4 + x_codeBlock3)/2)
        x_decodeBlock2 = decodeBlock2(self, (x_decodeBlock3 + x_codeBlock2)/2)
        x_decodeBlock1 = decodeBlock1(self, (x_decodeBlock2 + x_codeBlock1)/2)
        
        x_end = self.deconvDecodend(x_decodeBlock1)
        x_end = F.relu(self.convDecoddil1end(x_end))
        x_end = self.sigmoid(x_end)
        
        return(x_end)

In [ ]:
# This class takes our input of size 400*400 and directly put it to size 256*256

class FCN_DLinkNet_new(torch.nn.Module):
        
    def __init__(self):
        super(FCN_DLinkNet_new, self).__init__()
        
        # Encoding part
        
        self.convstart = torch.nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=57)
        self.pool = torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        
        self.convEncodBlock1 = torch.nn.Conv2d(64, 64, kernel_size=3, padding=1)
        
        self.convEncodBlock2start = torch.nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.convEncodBlock2 = torch.nn.Conv2d(128, 128, kernel_size=3, padding=1)
        
        self.convEncodBlock3start = torch.nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.convEncodBlock3 = torch.nn.Conv2d(256, 256, kernel_size=3, padding=1)
        
        self.convEncodBlock4start = torch.nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.convEncodBlock4 = torch.nn.Conv2d(512, 512, kernel_size=3, padding=1)
        
        # Center part
        
        self.convCenterdil1 = torch.nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, dilation=1) #same as convEncodBlock4 but added for clarity
        self.convCenterdil2 = torch.nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=2, dilation=2)
        self.convCenterdil4 = torch.nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=4, dilation=4)
        self.convCenterdil8 = torch.nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=8, dilation=8)
        
        # Decoding part
        
        self.convDecodBlock4start = torch.nn.Conv2d(512, 128, kernel_size=1)
        self.deconvDecodBlock4 = torch.nn.ConvTranspose2d(128, 128, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.convDecodBlock4end = torch.nn.Conv2d(128, 256, kernel_size=1)
        self.normback4 = torch.nn.BatchNorm2d(256)
        
        self.convDecodBlock3start = torch.nn.Conv2d(256, 64, kernel_size=1)
        self.deconvDecodBlock3 = torch.nn.ConvTranspose2d(64, 64, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.convDecodBlock3end = torch.nn.Conv2d(64, 128, kernel_size=1)
        self.normback3 = torch.nn.BatchNorm2d(128)
        
        self.convDecodBlock2start = torch.nn.Conv2d(128, 32, kernel_size=1)
        self.deconvDecodBlock2 = torch.nn.ConvTranspose2d(32, 32, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.convDecodBlock2end = torch.nn.Conv2d(32, 64, kernel_size=1)
        self.normback2 = torch.nn.BatchNorm2d(64)
        
        self.convDecodBlock1start = torch.nn.Conv2d(64, 16, kernel_size=1)
        self.deconvDecodBlock1 = torch.nn.ConvTranspose2d(16, 16, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.convDecodBlock1end = torch.nn.Conv2d(16, 64, kernel_size=1)
        self.normback1 = torch.nn.BatchNorm2d(64)
        
        self.deconvDecodend = torch.nn.ConvTranspose2d(64, 32, kernel_size=3, stride=1, padding=57)
        self.convDecoddil1end = torch.nn.Conv2d(32, 2, kernel_size=3, stride=1, padding=1, dilation=1)
        
        self.sigmoid = torch.nn.Sigmoid()
        
        
    def forward(self, x):
        
        def codeBlock1(self, x):
            for i in range(0, 3):
                x_inter = F.relu(self.convEncodBlock1(x))
                x_inter = F.relu(self.convEncodBlock1(x_inter))
                # Moyenne
                x = x + x_inter
            return x

        def codeBlock2(self, x):
            x = self.pool(x)
            x_inter = F.relu(self.convEncodBlock2start(x))
            x_inter = F.relu(self.convEncodBlock2(x_inter))
            new_x = torch.zeros(x_inter.shape)
            new_x[:, :64, :, :] = x
            # Moyenne
            x = new_x + x_inter
            for i in range(0, 3):
                x_inter = F.relu(self.convEncodBlock2(x))
                x_inter = F.relu(self.convEncodBlock2(x_inter))
                # Moyenne
                x = x + x_inter
            return x

        def codeBlock3(self, x):
            x = self.pool(x)
            x_inter = F.relu(self.convEncodBlock3start(x))
            x_inter = F.relu(self.convEncodBlock3(x_inter))
            new_x = torch.zeros(x_inter.shape)
            new_x[:, :128, :, :] = x
            # Moyenne
            x = new_x + x_inter
            for i in range(0, 5):
                x_inter = F.relu(self.convEncodBlock3(x))
                x_inter = F.relu(self.convEncodBlock3(x_inter))
                # Moyenne
                x = x + x_inter
            return x

        def codeBlock4(self, x):
            x = self.pool(x)
            x_inter = F.relu(self.convEncodBlock4start(x))
            x_inter = F.relu(self.convEncodBlock4(x_inter))
            new_x = torch.zeros(x_inter.shape)
            new_x[:, :256, :, :] = x
            # Moyenne
            x = new_x + x_inter
            for i in range(0, 2):
                x_inter = F.relu(self.convEncodBlock4(x))
                x_inter = F.relu(self.convEncodBlock4(x_inter))
                # Moyenne
                x = x + x_inter
            return x

        def decodeBlock4(self, x):
            x = F.relu(self.convDecodBlock4start(x))
            x = self.deconvDecodBlock4(x)
            x = F.relu(self.convDecodBlock4end(x))
            x = self.normback4(x)
            return x

        def decodeBlock3(self, x):
            x = F.relu(self.convDecodBlock3start(x))
            x = self.deconvDecodBlock3(x)
            x = F.relu(self.convDecodBlock3end(x))
            x = self.normback3(x)
            return x

        def decodeBlock2(self, x):
            x = F.relu(self.convDecodBlock2start(x))
            x = self.deconvDecodBlock2(x)
            x = F.relu(self.convDecodBlock2end(x))
            x = self.normback2(x)
            return x

        def decodeBlock1(self, x):
            x = F.relu(self.convDecodBlock1start(x))
            x = self.deconvDecodBlock1(x)
            x = F.relu(self.convDecodBlock1end(x))
            x = self.normback1(x)
            return x
        
        x_start = F.relu(self.convstart(x))
        
        x_codeBlock1 = codeBlock1(self, x_start)
        x_codeBlock2 = codeBlock2(self, x_codeBlock1)
        x_codeBlock3 = codeBlock3(self, x_codeBlock2)
        x_codeBlock4 = codeBlock4(self, x_codeBlock3)
        
        x_center1 = F.relu(self.convCenterdil1(x_codeBlock4))
        x_center2 = F.relu(self.convCenterdil2(x_center1))
        x_center3 = F.relu(self.convCenterdil4(x_center2))
        x_center4 = F.relu(self.convCenterdil8(x_center3))
        
        x_center_final = x_codeBlock4 + x_center1 + x_center2 + x_center3 + x_center4
        
        x_decodeBlock4 = decodeBlock4(self, x_center_final)
        x_decodeBlock3 = decodeBlock3(self, x_decodeBlock4 + x_codeBlock3)
        x_decodeBlock2 = decodeBlock2(self, x_decodeBlock3 + x_codeBlock2)
        x_decodeBlock1 = decodeBlock1(self, x_decodeBlock2 + x_codeBlock1)
        
        x_end = self.deconvDecodend(x_decodeBlock1)
        x_end = F.relu(self.convDecoddil1end(x_end))
        x_end = self.sigmoid(x_end)
        
        return(x_end)

In [ ]:
# This class takes our input of size 400*400 and directly put it to size 256*256

class FCN_DLinkNet_direct(torch.nn.Module):
        
    def __init__(self):
        super(FCN_DLinkNet_direct, self).__init__()
        
        # Encoding part
        
        self.convstart = torch.nn.Conv2d(3, 64, kernel_size=5, stride=1, padding=2)
        self.pool = torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.convEncodBlock1 = torch.nn.Conv2d(64, 64, kernel_size=3, padding=1)
        
        self.convEncodBlock2start = torch.nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.convEncodBlock2 = torch.nn.Conv2d(128, 128, kernel_size=3, padding=1)
        
        self.convEncodBlock3start = torch.nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.convEncodBlock3 = torch.nn.Conv2d(256, 256, kernel_size=3, padding=1)
        
        self.convEncodBlock4start = torch.nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.convEncodBlock4 = torch.nn.Conv2d(512, 512, kernel_size=3, padding=1)
        
        # Center part
        
        self.convCenterdil1 = torch.nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, dilation=1) #same as convEncodBlock4 but added for clarity
        self.convCenterdil2 = torch.nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=2, dilation=2)
        self.convCenterdil4 = torch.nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=4, dilation=4)
        self.convCenterdil8 = torch.nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=8, dilation=8)
        
        # Decoding part
        
        self.convDecodBlock4start = torch.nn.Conv2d(512, 128, kernel_size=1)
        self.deconvDecodBlock4 = torch.nn.ConvTranspose2d(128, 128, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.convDecodBlock4end = torch.nn.Conv2d(128, 256, kernel_size=1)
        
        self.convDecodBlock3start = torch.nn.Conv2d(256, 64, kernel_size=1)
        self.deconvDecodBlock3 = torch.nn.ConvTranspose2d(64, 64, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.convDecodBlock3end = torch.nn.Conv2d(64, 128, kernel_size=1)
        
        self.convDecodBlock2start = torch.nn.Conv2d(128, 32, kernel_size=1)
        self.deconvDecodBlock2 = torch.nn.ConvTranspose2d(32, 32, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.convDecodBlock2end = torch.nn.Conv2d(32, 64, kernel_size=1)
        
        self.convDecodBlock1start = torch.nn.Conv2d(64, 16, kernel_size=1)
        self.deconvDecodBlock1 = torch.nn.ConvTranspose2d(16, 16, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.convDecodBlock1end = torch.nn.Conv2d(16, 64, kernel_size=1)
        
        self.deconvDecodend = torch.nn.ConvTranspose2d(64, 32, kernel_size=3, stride=1, padding=1)
        self.convDecoddil1end = torch.nn.Conv2d(32, 2, kernel_size=3, stride=2, padding=2, dilation=2)
        
        self.sigmoid = torch.nn.Sigmoid()
        
        
    def forward(self, x):
        
        def codeBlock1(self, x):
            for i in range(0, 3):
                x_inter = F.relu(self.convEncodBlock1(x))
                x_inter = F.relu(self.convEncodBlock1(x_inter))
                # Moyenne
                x = x + x_inter
            return x

        def codeBlock2(self, x):
            x = self.pool(x)
            x_inter = F.relu(self.convEncodBlock2start(x))
            x_inter = F.relu(self.convEncodBlock2(x_inter))
            new_x = torch.zeros(x_inter.shape)
            new_x[:, :64, :, :] = x
            # Moyenne
            x = new_x + x_inter
            for i in range(0, 3):
                x_inter = F.relu(self.convEncodBlock2(x))
                x_inter = F.relu(self.convEncodBlock2(x_inter))
                # Moyenne
                x = x + x_inter
            return x

        def codeBlock3(self, x):
            x = self.pool(x)
            x_inter = F.relu(self.convEncodBlock3start(x))
            x_inter = F.relu(self.convEncodBlock3(x_inter))
            new_x = torch.zeros(x_inter.shape)
            new_x[:, :128, :, :] = x
            # Moyenne
            x = new_x + x_inter
            for i in range(0, 5):
                x_inter = F.relu(self.convEncodBlock3(x))
                x_inter = F.relu(self.convEncodBlock3(x_inter))
                # Moyenne
                x = x + x_inter
            return x

        def codeBlock4(self, x):
            x = self.pool(x)
            x_inter = F.relu(self.convEncodBlock4start(x))
            x_inter = F.relu(self.convEncodBlock4(x_inter))
            new_x = torch.zeros(x_inter.shape)
            new_x[:, :256, :, :] = x
            # Moyenne
            x = new_x + x_inter
            for i in range(0, 2):
                x_inter = F.relu(self.convEncodBlock4(x))
                x_inter = F.relu(self.convEncodBlock4(x_inter))
                # Moyenne
                x = x + x_inter
            return x

        def decodeBlock4(self, x):
            x = F.relu(self.convDecodBlock4start(x))
            x = self.deconvDecodBlock4(x)
            x = F.relu(self.convDecodBlock4end(x))
            return x

        def decodeBlock3(self, x):
            x = F.relu(self.convDecodBlock3start(x))
            x = self.deconvDecodBlock3(x)
            x = F.relu(self.convDecodBlock3end(x))
            return x

        def decodeBlock2(self, x):
            x = F.relu(self.convDecodBlock2start(x))
            x = self.deconvDecodBlock2(x)
            x = F.relu(self.convDecodBlock2end(x))
            return x

        def decodeBlock1(self, x):
            x = F.relu(self.convDecodBlock1start(x))
            x = self.deconvDecodBlock1(x)
            x = F.relu(self.convDecodBlock1end(x))
            return x
        
        x_start = F.relu(self.convstart(x))
        
        x_codeBlock1 = codeBlock1(self, x_start)
        x_codeBlock2 = codeBlock2(self, x_codeBlock1)
        x_codeBlock3 = codeBlock3(self, x_codeBlock2)
        x_codeBlock4 = codeBlock4(self, x_codeBlock3)
        
        x_center1 = F.relu(self.convCenterdil1(x_codeBlock4))
        x_center2 = F.relu(self.convCenterdil2(x_center1))
        x_center3 = F.relu(self.convCenterdil4(x_center2))
        x_center4 = F.relu(self.convCenterdil8(x_center3))
        
        x_center_final = x_codeBlock4 + x_center1 + x_center2 + x_center3 + x_center4
        
        x_decodeBlock4 = decodeBlock4(self, x_center_final)
        x_decodeBlock3 = decodeBlock3(self, x_decodeBlock4 + x_codeBlock3)
        x_decodeBlock2 = decodeBlock2(self, x_decodeBlock3 + x_codeBlock2)
        x_decodeBlock1 = decodeBlock1(self, x_decodeBlock2 + x_codeBlock1)
        
        x_end = self.deconvDecodend(x_decodeBlock1)
        x_end = F.relu(self.convDecoddil1end(x_end))
        x_end = self.sigmoid(x_end)
        
        return(x_end)

This model optimized with adam and cross entropy will converge to all-black images every time

## Training the model

In [ ]:
end_train = 15 # normally = 90
end_validation = 20 # normally = 100

# Loading a set of 100 training images
root_dir = "training/"

image_dir = root_dir + "images/"
files = os.listdir(image_dir)
n = min(100, len(files)) # Load maximum 100 images
print("Loading " + str(n) + " images")
imgs = np.array([load_image(image_dir + files[i]) for i in range(n)]).swapaxes(1,3).swapaxes(2,3)
print(np.shape(imgs))

train_input = imgs[0:end_train] #normally = 0:90
validation_input = imgs[end_train:end_validation] #normally = 90:100

image_dir = root_dir + "groundtruth/"
files = os.listdir(image_dir)
n = min(100, len(files)) # Load maximum 100 images
print("Loading " + str(n) + " images")
grounds = [load_image(image_dir + files[i]) for i in range(n)]
print(np.shape(grounds))

train_target = grounds[0:end_train] #normally = 0:90
validation_target = grounds[end_train:end_validation] #normally = 90:100

In [ ]:
# crop images to their 256*256 counterparts
cropped_imgs = []
cropped_targets = []

for i in range(end_validation):
    cropped_img, k, l = img_rnd_crop(imgs[i], 256, 256)
    cropped_target, _, _ = img_rnd_crop(grounds[i], 256, 256, k, l)
    cropped_imgs.append(cropped_img)
    cropped_targets.append(cropped_target)
    
train_input = cropped_imgs[0:end_train] #normally = 0:90
validation_input = cropped_imgs[end_train:end_validation] #normally = 90:100


train_target = cropped_targets[0:end_train] #normally = 0:90
validation_target = cropped_targets[end_train:end_validation] #normally = 90:100

display(np.shape(train_input))
display(np.shape(validation_input))
display(np.shape(train_target))
display(np.shape(validation_target))

We will keep 10 images from this set as a validation set.
To shorten the computationnal time we will use a smaller amount of images to do tests.

### Instantiate the model, loss function and optimizer

In [ ]:
# We will optimize the cross-entropy loss using adam algorithm
net = FCN_DLinkNet_direct()
loss_function = torch.nn.CrossEntropyLoss()
# optimizer = optim.Adam(net.parameters(), lr=0.001)
optimizer = optim.SGD(net.parameters(), lr=3.75e-2, momentum=0.9)

In [ ]:
def trainNet(net, n_epochs):
    
    #Time for printing
    training_start_time = time.time()
    
    #Loop for n_epochs
    for epoch in range(n_epochs):
        
        total_loss = 0.0
        
        for index in range(np.shape(train_input)[0]):
            
            input_image = Variable(torch.tensor(train_input[index], requires_grad=True).unsqueeze(0))
            target_image = Variable(torch.tensor(train_target[index], dtype=torch.long).unsqueeze(0))
            
            #Set the parameter gradients to zero
            optimizer.zero_grad()
            
            #Forward pass, backward pass, optimize
            outputs = net(input_image)
            loss = loss_function(outputs, target_image)
            loss.backward()
            optimizer.step()
            
            #Print statistics
            total_loss += loss.item()
            
            print("Epoch", epoch, ", image", index, ", image loss:", loss.item(), ", time elapsed:", time.time() - training_start_time)
            
        #At the end of the epoch, do a pass on the validation set
        total_val_loss = 0
        for index in range(np.shape(validation_input)[0]):
            
            input_image = Variable(torch.tensor(validation_input[index], requires_grad=True).unsqueeze(0))
            target_image = Variable(torch.tensor(validation_target[index], dtype=torch.long).unsqueeze(0))
            
            #Forward pass
            val_outputs = net(input_image)
            val_loss = loss_function(val_outputs, target_image)
            total_val_loss += val_loss.item()
            
        print("Validation loss for epoch", epoch, ":", total_val_loss/np.shape(validation_input)[0])
        
    print("Training finished, took {:.2f}s".format(time.time() - training_start_time))

In [ ]:
trainNet(net, 3)

In [ ]:
input_image = torch.tensor(validation_input[0]).unsqueeze(0)
target_image = torch.tensor(validation_target[0]).unsqueeze(0)
           
#Forward pass
val_output = net(input_image)
output_image = val_output[0,1]

In [ ]:
plt.imshow(target_image.squeeze(0), cmap='Greys_r')

In [ ]:
plt.imshow(output_image.detach().numpy(), cmap='Greys_r')